In [92]:
# Given the text and corresponding evidence, identify and flag any factual errors. Return 1 if very likely factual errors are found, or return 0 if very likely no factual errors are found. Only provide the number without any additional information.

# Text: {text}
# Evidence: {evidence}



# Identify and flag any factual errors in the given text and corresponding evidence. Return 1 if factual errors are found, or return 0 if no factual errors are found.Only provide the number without any additional information.

# Text: {text}
# Evidence: {evidence}



# Given a piece of text and corresponding evidence, your task is to identify and flag any factual errors in the text. If the text is factually accurate based on the provided evidence, return a value between 0 and 1 (inclusive) in increments of 0.2, representing the level of confidence in its accuracy. It is sufficient to return a single number without any other information.

# 0: No factual errors found.
# 0.2: Slight doubts about the accuracy of the text.
# 0.4: Some factual errors detected.
# 0.6: Moderate doubts about the accuracy of the text.
# 0.8: Significant factual errors identified.
# 1: Text contains multiple factual errors.

# Text: {text}
# Evidence: {evidence}



# Given a piece of text and corresponding evidence, your task is to identify and flag any factual errors in the text. If the text is factually accurate based on the provided evidence, return a confidence score between 0 and 1. A score closer to 1 indicates the presence of multiple factual errors, while a score closer to 0 indicates no factual errors found. Provide only the confidence score without additional information.

# Text: {text}
# Evidence: {evidence}

In [13]:
instruction_mt='This is a machine translation task. Given a standard translation, and a model output translation, determine if the model output is subject to hallucination.\n\
{demo}\n\n\
your task:\n\
standard translation: {Evidence}\n\
model output translation: {Text}\n\n\
The criteria for judging are as follows:\n\
Check if the model output translation is fluent and answers the question.\n\
Compare the model output translation with correct examples. If inconsistencies are found or it can\'t be inferred from the standard translation, it\'s likely hallucination.\n\
If the model output translation aligns with the standard translation or has a similar meaning, it\'s likely not hallucination.\n\
If the standard translation is "unanswerable" and the model output translation is "I don\'t know," it\'s likely not hallucination.\n\
please only return 0 or 1. Return 1 for hallucination; return 0 for not hallucination.'


instruction_dm='This is a definition modeling task. Given a standard definition of a word, and a model output definition of this word, determine if the model output is subject to hallucination.\n\
{demo}\n\n\
your task:\n\
standard definition: {Evidence}\n\
model output definition: {Text}\n\n\
The criteria for judging are as follows:\n\
Check if the model output definition is fluent and answers the question.\n\
Compare the model output definition with correct examples. If inconsistencies are found or it can\'t be inferred from the standard definition, it\'s likely hallucination.\n\
If the model output definition aligns with the standard definition or has a similar meaning, it\'s likely not hallucination.\n\
If the standard definition is "unanswerable" and the model output definition is "I don\'t know," it\'s likely not hallucination.\n\
please only return 0 or 1. Return 1 for hallucination; return 0 for not hallucination.'


instruction_pg='This is a paraphrase generation task, which transforms a original sentence into a new sentence. Given a original sentence, and a model output new sentence, determine if the model output is subject to hallucination.\n\
{demo}\n\n\
your task:\n\
original sentence: {Evidence}\n\
model output new sentence: {Text}\n\n\
The criteria for judging are as follows:\n\
Check if the model output new sentence is fluent and answers the question.\n\
Compare the model output new sentence with correct examples. If inconsistencies are found or it can\'t be inferred from the original sentence, it\'s likely hallucination.\n\
If the model output new sentence aligns with the original sentence or has a similar meaning, it\'s likely not hallucination.\n\
If the original sentence is "unanswerable" and the model output new sentence is "I don\'t know," it\'s likely not hallucination.\n\
please only return 0 or 1. Return 1 for hallucination; return 0 for not hallucination.'

In [18]:
import pandas as pd
import numpy as np
import re
import time
from openai import OpenAI


client = OpenAI()

# df=pd.read_json(r'C:\Users\刘威\Desktop\SHROOM\SHROOM_dev-v1\val.model-agnostic.json')
df=pd.read_json(r'C:\Users\刘威\Desktop\SHROOM\unlabeled-training-data\train.model-agnostic.json',lines=False)
# df = pd.read_json(r'C:\Users\刘威\Desktop\SHROOM\SHROOM_dev-v1\val.model-aware-selectdemo1.json', lines=True)

# ['hyp', 'ref', 'src', 'tgt', 'model', 'task', 'labels', 'label', 'p(Hallucination)',
# 'hyptop10sbert', 'srctop10sbert', 'tgttop10sbert', 'hyptop10bm25', 'srctop10bm25', 'tgttop10bm25']

df = df[['tgt', 'src', 'hyp', 'task']]
df.columns = ['tgt', 'src', 'Text', 'task']
df['label_pre'] = ''

# df = df.sample(30,random_state=40)
# df.index=np.arange(df.shape[0])

In [19]:
df1=df[df['task']=='DM']
df1 = df1.sample(1000,random_state=42)

df2=df[df['task']=='PG']
df2 = df2.sample(1000,random_state=42)

df3=df[df['task']=='MT']
df3 = df3.sample(750,random_state=42)

df = pd.concat([df1, df2, df3])
df.index = np.arange(df.shape[0])
df['task'].value_counts()

task
DM    1000
PG    1000
MT     750
Name: count, dtype: int64

In [20]:
df['tgt'] = df['tgt'].apply(lambda x: re.sub(r"\s'", "'", x))
df['tgt'] = df['tgt'].apply(lambda x: re.sub(r"\s;", ";", x))
df['tgt'] = df['tgt'].apply(lambda x: re.sub(r"\s:", ":", x))
df['tgt'] = df['tgt'].apply(lambda x: re.sub(r"\s\.", ".", x))

df['src'] = df['src'].apply(lambda x: re.sub(r"\s'", "'", x))
df['src'] = df['src'].apply(lambda x: re.sub(r"\s;", ";", x))
df['src'] = df['src'].apply(lambda x: re.sub(r"\s:", ":", x))
df['src'] = df['src'].apply(lambda x: re.sub(r"\s\.", ".", x))

df['Text'] = df['Text'].apply(lambda x: re.sub(r"\s'", "'", x))
df['Text'] = df['Text'].apply(lambda x: re.sub(r"\s;", ";", x))
df['Text'] = df['Text'].apply(lambda x: re.sub(r"\s:", ":", x))
df['Text'] = df['Text'].apply(lambda x: re.sub(r"\s\.", ".", x))

In [ ]:
# DM:tgt   MT:tgt   PG:src

for i in np.arange(df.shape[0]):

    Text = df.loc[i, 'Text']

    if df.loc[i, 'task'] == 'DM':
        Evidence_column = 'tgt'
        Evidence = df.loc[i, 'tgt']
        prompt_instruction = instruction_dm
        demo_count = 0  # number of demos
        demo_select = 'tgttop10bm25'  # Demo selection method
        Evidence_format='standard definition: '
        Text_format='model output definition: '
    elif df.loc[i, 'task'] == 'MT':
        Evidence_column = 'tgt'
        Evidence = df.loc[i, 'tgt']
        prompt_instruction = instruction_mt
        demo_count = 0  # number of demos
        demo_select = 'tgttop10bm25'  # Demo selection method
        Evidence_format='standard translation: '
        Text_format='model output translation: '
    elif df.loc[i, 'task'] == 'PG':
        Evidence_column = 'src'
        Evidence = df.loc[i, 'src']
        prompt_instruction = instruction_pg
        demo_count = 0  # number of demos
        demo_select = 'srctop10bm25'  # Demo selection method
        Evidence_format='original sentence: '
        Text_format='model output new sentence: '
    else:
        print('error')
        break

    if demo_count == 0:
        demo = ''
    else:
        demo = ''
        demo_indexs = [index for index in eval(df.loc[i, demo_select])][:demo_count]
        demo_index_count = 1
        for demo_index in demo_indexs:
            demo = demo + '\nExample ' + str(demo_index_count) + ':'
            demo_index_count = demo_index_count + 1
            # demo = demo + "\n" + Text_format + df.loc[demo_index, 'Text'] + "\n" + Evidence_format + \
            #     df.loc[demo_index, Evidence_column] + "\nOutput: " + str(df.loc[demo_index, 'label'])
            demo = demo + "\n" + Evidence_format + df.loc[demo_index, Evidence_column] + "\n" + Text_format + \
                df.loc[demo_index, 'Text'] + "\nOutput: " + str(df.loc[demo_index, 'label'])
            
        demo = demo + "\n\nNext Task:\n"

    # content = prompt_instruction + "\n" + demo + "Text: " + Text + "\nEvidence: " + Evidence
    content = prompt_instruction
    content = content.format(demo=demo,Evidence=Evidence,Text=Text)
    

    
    # content = content + '\nOnly provide 0 or 1 without any additional information.'

    # print(content)

    while True:
        try:
            response = client.chat.completions.create(
                model="gpt-4-1106-preview",
                messages=[{"role": "user","content": content}],
                temperature=1.0,
                max_tokens=400,
                n=5
            )

            answer = [response.choices[i].message.content for i in range(len(response.choices))]
            # answer = response["choices"][0]["message"]["content"]

            df.loc[i, 'label_pre'] = str(answer)

            # print('第', i, '条数据:   ', df.loc[i, 'label_pre'])

            time.sleep(0.2)

            break
        except Exception as e:
            print(e)
            time.sleep(5)

In [ ]:
df.head()

In [23]:
df.columns = ['tgt', 'src', 'hyp', 'task', 'label_pre']

In [24]:
df_json_str = df.to_json(orient='records', force_ascii=False, lines=True)
with open('label2500_train.model-agnostic.json', 'w', encoding='utf-8') as f:
    f.write(df_json_str)

In [13]:
import pandas as pd
from collections import Counter
import json

df = pd.read_json('label2500_train.model-agnostic.json', lines=True)

for index, row in df.iterrows():
    try:
        s = row["label_pre"].replace("'", '"')
        labels = json.loads(s)
        labels = [int(i) for i in labels if i in ['0','1']]

        if not labels:
            print("wrong")
            df.loc[index, 'label'] = 1
        else:
            label_counts = Counter(labels)
            mode_label = label_counts.most_common(1)[0][0]

            df.loc[index, 'label'] = mode_label
    except Exception:
        print("***********",index)
        df.loc[index, 'label'] = 1

df['label'] = df['label'].apply(lambda x: int(x))

df_json_str = df.to_json(orient='records', force_ascii=False, lines=True)
with open('labeled_data.json', 'w', encoding='utf-8') as f:
    f.write(df_json_str)

*********** 748
*********** 1527
*********** 1708
*********** 1784
*********** 1792
